In [1]:
from sklearn import tree
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
import zipfile
from statsmodels.tsa.deterministic import DeterministicProcess, CalendarFourier
from statsmodels.graphics.tsaplots import plot_pacf
from sklearn.preprocessing import RobustScaler, StandardScaler, Normalizer, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, BaggingRegressor
from sklearn.model_selection import train_test_split

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import mean_absolute_error

In [16]:
zf = zipfile.ZipFile('C:/Users/Serueja/Downloads/house-prices-advanced-regression-techniques.zip')
test = pd.read_csv(zf.open('test.csv'))
train = pd.read_csv(zf.open('train.csv'))
X = train.drop('SalePrice', axis=1)
X.fillna(value=0, inplace=True)
y = train['SalePrice']
submission_sample = pd.read_csv(zf.open('sample_submission.csv'))

In [24]:
Z_test = test.copy()
Z_test = pd.get_dummies(Z_test)
Z_test.fillna(value=0, inplace=True)

In [26]:
difference = set(X.columns).difference(set(Z_test.columns))

In [27]:
X = pd.get_dummies(X)
X.drop(list(difference), axis=1, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [18]:
model_random_forest = RandomForestClassifier()
parameters = {'criterion': ['log_loss'], 'max_depth': range(1, 100)}
grid_search_cv_clf=GridSearchCV(model_random_forest, parameters, cv=5)

In [19]:
grid_search_cv_clf.fit(X_train, y_train)

c:\Users\Serueja\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['log_loss'], 'max_depth': range(1, 100)})

In [20]:
print(grid_search_cv_clf.best_params_)
best_max_depth_random_forest_params = grid_search_cv_clf.best_params_
best_max_depth_random_forest_model = grid_search_cv_clf.best_estimator_
d = {'PredictedSales':best_max_depth_random_forest_model.predict(X_test), 'ActualSales':y_test}
predicted_actual_df=pd.DataFrame(data=d, index=np.arange(len(y_test.index)))

{'criterion': 'log_loss', 'max_depth': 7}


In [28]:
temp_model = RandomForestClassifier(criterion='log_loss', max_depth=7)

In [30]:
temp_model.fit(X_train, y_train)

RandomForestClassifier(criterion='log_loss', max_depth=7)

In [31]:
temp_model.score(X_train, y_train)
y_pred = temp_model.predict(X_test)
precision_score(y_pred, y_test, average='weighted', zero_division=1)

0.3619634703196347

In [22]:
best_max_depth_random_forest_model.score(X_train, y_train)
y_pred = best_max_depth_random_forest_model.predict(X_test)
precision_score(y_pred, y_test, average='weighted', zero_division=1)

0.31689497716894977

In [33]:
super_prediction=temp_model.predict(Z_test)
super_prediction=pd.Series(super_prediction)
sub = pd.DataFrame({'Id':test['Id'], 'SalePrice':super_prediction})
sub.to_csv('submission.csv', index=False)